<a href="https://colab.research.google.com/github/markusstrasser/deforum/blob/main/Deforum_Stable_Diffusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Deforum Stable Diffusion v0.5**
[Stable Diffusion](https://github.com/CompVis/stable-diffusion) by Robin Rombach, Andreas Blattmann, Dominik Lorenz, Patrick Esser, Björn Ommer and the [Stability.ai](https://stability.ai/) Team. [K Diffusion](https://github.com/crowsonkb/k-diffusion) by [Katherine Crowson](https://twitter.com/RiversHaveWings). You need to get the ckpt file and put it on your Google Drive first to use this. It can be downloaded from [HuggingFace](https://huggingface.co/CompVis/stable-diffusion).

Notebook by [deforum](https://discord.gg/upmXXsrwZc)

By using this Notebook, you agree to the following Terms of Use, and license:

**Stablity.AI Model Terms of Use**

This model is open access and available to all, with a CreativeML OpenRAIL-M license further specifying rights and usage.

The CreativeML OpenRAIL License specifies:

You can't use the model to deliberately produce nor share illegal or harmful outputs or content
CompVis claims no rights on the outputs you generate, you are free to use them and are accountable for their use which must not go against the provisions set in the license
You may re-distribute the weights and use the model commercially and/or as a service. If you do, please be aware you have to include the same use restrictions as the ones in the license and share a copy of the CreativeML OpenRAIL-M to all your users (please read the license entirely and carefully)


Please read the full license here: https://huggingface.co/spaces/CompVis/stable-diffusion-license

# Setup

In [1]:
!rm -rf stable-diffusion/
!rm -rf src
!rm -rf pytorch3d-lite/
!rm -rf MiDaS
!rm -rf k-diffusion
!rm -rf AdaBins

#@markdown **NVIDIA GPU**
import subprocess
sub_p_res = subprocess.run(['nvidia-smi', '--query-gpu=name,memory.total,memory.free', '--format=csv,noheader'], stdout=subprocess.PIPE).stdout.decode('utf-8')
print(sub_p_res)



setup_environment = True #@param {type:"boolean"}
print_subprocess = False #@param {type:"boolean"}

if setup_environment:
    import subprocess, time
    print("Setting up environment...")
    start_time = time.time()
    all_process = [
        ['pip', 'install', 'torch==1.12.1+cu113', 'torchvision==0.13.1+cu113', '--extra-index-url', 'https://download.pytorch.org/whl/cu113'],
        ['pip', 'install', 'omegaconf==2.2.3', 'einops==0.4.1', 'pytorch-lightning==1.7.4', 'torchmetrics==0.9.3', 'torchtext==0.13.1', 'transformers==4.21.2', 'kornia==0.6.7'],
        ['git', 'clone',  'https://github.com/deforum/stable-diffusion'],
        ['pip', 'install', '-e', 'git+https://github.com/CompVis/taming-transformers.git@master#egg=taming-transformers'],
        ['pip', 'install', '-e', 'git+https://github.com/openai/CLIP.git@main#egg=clip'],
        ['pip', 'install', 'accelerate', 'ftfy', 'jsonmerge', 'matplotlib', 'resize-right', 'timm', 'torchdiffeq'],
        ['git', 'clone', 'https://github.com/shariqfarooq123/AdaBins.git'],
        ['git', 'clone', 'https://github.com/isl-org/MiDaS.git'],
        ['git', 'clone', 'https://github.com/MSFTserver/pytorch3d-lite.git'],
    ]
    for process in all_process:
        running = subprocess.run(process,stdout=subprocess.PIPE).stdout.decode('utf-8')
        if print_subprocess:
            print(running)
    
    print(subprocess.run(['git', 'clone', 'https://github.com/deforum/k-diffusion/'], stdout=subprocess.PIPE).stdout.decode('utf-8'))
    with open('k-diffusion/k_diffusion/__init__.py', 'w') as f:
        f.write('')

    end_time = time.time()
    print(f"Environment set up in {end_time-start_time:.0f} seconds")

NVIDIA RTX A5000, 24564 MiB, 24255 MiB

Setting up environment...


Cloning into 'stable-diffusion'...
  Running command git clone -q https://github.com/CompVis/taming-transformers.git /workspace/deforum/src/taming-transformers
  Running command git clone -q https://github.com/openai/CLIP.git /workspace/deforum/src/clip
Cloning into 'AdaBins'...
Cloning into 'MiDaS'...
Cloning into 'pytorch3d-lite'...
Cloning into 'k-diffusion'...



Environment set up in 119 seconds


In [2]:
!pip install gdown
!gdown https://drive.google.com/uc?id=1N06uaEC-5c1ciC_CVwqfhD-IPbOtCVOa
!mkdir ../models
!mv sd-v1-4-full-ema.ckpt ../models/sd-v1-4-full-ema.ckpt
!pip install numexpr pandas scikit-image wget opencv-python

!apt-get update && apt-get install libgl1 --yes
!apt-get install libglib2.0-0 --yes
!apt update
!apt install wget

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for gdown: filename=gdown-4.5.1-py3-none-any.whl size=14933 sha256=2ecb51d12e64965e7711cbe80275d4643e59744e3313855d622f06e9f1b2ff04
  Stored in directory: /root/.cache/pip/wheels/3d/ec/b0/a96d1d126183f98570a785e6bf8789fca559853a9260e928e1
Successfully built gdown
Downloading...
From: https://drive.google.com/uc?id=1N06uaEC-5c1ciC_CVwqfhD-IPbOtCVOa
To: /workspace/deforum/sd-v1-4-full-ema.ckpt
100%|██████████████████████████████████████| 7.70G/7.70G [01:51<00:00, 68.8MB/s]
     |████████████████████████████████| 379 kB 6.5 MB/s eta 0:00:01
     |████████████████████████████████| 11.3 MB 45.6 MB/s eta 0:00:01
     |████████████████████████████████| 13.5 MB 33.3 MB/s eta 0:00:01
     |████████████████████████████████| 6.4 MB 30.0 MB/s eta 0:00:01
     |████████████████████████████████| 3.4 MB 44.9 MB/s eta 0:00:01
     |██████████████████████████████

# Settings

### Animation Settings

# Run

In [4]:
pip install opencv-python

     |████████████████████████████████| 60.9 MB 4.6 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [5]:
%load_ext autoreload
%autoreload 2
from deforum import render_input_video, render_animation, render_interpolation, render_image_batch, DeforumArgs, DeforumAnimArgs

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Local Path Variables:

models_path: ../models/
output_path: /content/output
Using config: ./stable-diffusion/configs/stable-diffusion/v1-inference.yaml
../models/sd-v1-4-full-ema.ckpt modelpath
../models/sd-v1-4-full-ema.ckpt exists
Using ckpt: ../models/sd-v1-4-full-ema.ckpt
Loading model from ../models/sd-v1-4-full-ema.ckpt
Global Step: 470000
LatentDiffusion: Running in eps-prediction mode
DiffusionWrapper has 859.52 M params.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 32, 32) = 4096 dimensions.
making attention of type 'vanilla' with 512 in_channels


Some weights of the model checkpoint at openai/clip-vit-large-patch14 were not used when initializing CLIPTextModel: ['vision_model.encoder.layers.8.self_attn.k_proj.weight', 'vision_model.encoder.layers.10.layer_norm2.weight', 'vision_model.encoder.layers.13.mlp.fc1.bias', 'vision_model.encoder.layers.7.self_attn.q_proj.bias', 'vision_model.encoder.layers.15.mlp.fc1.weight', 'vision_model.encoder.layers.4.layer_norm2.bias', 'vision_model.encoder.layers.20.mlp.fc1.bias', 'vision_model.encoder.layers.6.mlp.fc1.bias', 'vision_model.encoder.layers.9.mlp.fc2.weight', 'vision_model.encoder.layers.10.self_attn.out_proj.weight', 'vision_model.encoder.layers.8.self_attn.out_proj.weight', 'vision_model.encoder.layers.13.layer_norm1.weight', 'vision_model.encoder.layers.13.mlp.fc2.weight', 'vision_model.encoder.layers.11.self_attn.out_proj.weight', 'vision_model.encoder.layers.0.self_attn.q_proj.bias', 'vision_model.encoder.layers.23.self_attn.v_proj.bias', 'vision_model.encoder.layers.0.self_at

In [30]:

import gc, torch
# clean up unused memory
gc.collect()
torch.cuda.empty_cache()

import os
from types import SimpleNamespace
import time, random, json
from datetime import datetime
from math import floor

#@markdown **Load Settings**
override_settings_with_file = True #@param {type:"boolean"}
custom_settings_file = "configs/current.txt"#@param {type:"string"}


# Getting the current date and time


args_global = {}

def run(e={}):
    args_dict = DeforumArgs()
    anim_args_dict = DeforumAnimArgs()
    args = {}
    args["prompts"] = {}

    if override_settings_with_file:
        print(f"reading custom settings from {custom_settings_file}")
        if not os.path.isfile(custom_settings_file):
            print('The custom settings file does not exist. The in-notebook settings will be used instead')
        else:
            with open(custom_settings_file, "r") as f:
                jdata = json.loads(f.read())
                jdata = {**jdata, **e}
                animation_prompts = jdata["prompts"]
                animation_prompts = {int(k): v for k,v in animation_prompts.items()}
                # args.prompts = animation_prompts
                for i, k in enumerate(args_dict):
                    if k in jdata:
                        args_dict[k] = jdata[k]
                    else:
                        print(f"key {k} doesn't exist in the custom settings data! using the default value of {args_dict[k]}")
                for i, k in enumerate(anim_args_dict):
                    if k in jdata:
                        anim_args_dict[k] = jdata[k]
                    else:
                        print(f"key {k} doesn't exist in the custom settings data! using the default value of {anim_args_dict[k]}")



                print(args_dict)
                print(anim_args_dict)


    print("animaattttt", animation_prompts, args)
    args = SimpleNamespace(**args_dict)
    anim_args = SimpleNamespace(**anim_args_dict)

    args.timestring = time.strftime('%Y%m%d%H%M%S')
    args.strength = max(0.0, min(1.0, args.strength))

    if args.seed == -1:
        args.seed = random.randint(0, 2**32 - 1)
    if not args.use_init:
        args.init_image = None
    if args.sampler == 'plms' and (args.use_init or anim_args.animation_mode != 'None'):
        print(f"Init images aren't supported with PLMS yet, switching to KLMS")
        args.sampler = 'klms'
    if args.sampler != 'ddim':
        args.ddim_eta = 0

    if anim_args.animation_mode == 'None':
        anim_args.max_frames = 1
    elif anim_args.animation_mode == 'Video Input':
        args.use_init = True
    
    args_global = args
    # clean up unused memory
    gc.collect()
    torch.cuda.empty_cache()

    # dispatch to appropriate renderer
    if anim_args.animation_mode == '2D' or anim_args.animation_mode == '3D':
        render_animation(args, anim_args, animation_prompts)
    elif anim_args.animation_mode == 'Video Input':
        render_input_video(args, anim_args, animation_prompts)
    elif anim_args.animation_mode == 'Interpolation':
        render_interpolation(args, anim_args, animation_prompts)
    else:
        render_image_batch(args)
    
    args.max_frames = anim_args.max_frames
    make_video(args)

In [31]:
import cv2
import numpy as np
import glob


def make_video(args):
    skip_video_for_run_all = False #@param {type: 'boolean'}
    fps = 12 #@param {type:"number"}
    #@markdown **Manual Settings**
    use_manual_settings = False #@param {type:"boolean"}
    image_path = "/content/drive/MyDrive/AI/StableDiffusion/2022-09/beyonce/20220928090240_%05d.png" #@param {type:"string"}
    mp4_path = "/content/drive/MyDrive/AI/StableDiffusion/2022-09/20220928090240.mp4" #@param {type:"string"}


    FRAME_RATE = 18


    if skip_video_for_run_all == True:
        print('Skipping video creation, uncheck skip_video_for_run_all if you want to run it')
    else:
        import os
        import subprocess
        from base64 import b64encode


        if use_manual_settings:
            max_frames = "500" #@param {type:"string"}
        else:
            # image_path = os.path.join(args.outdir, f"{args.timestring}_%05d.png")
            image_path = os.path.join(args.outdir, f"{args.timestring}_*.png")
            mp4_path = os.path.join(args.outdir, f"v_{args.timestring}.mp4")
            print(mp4_path, "pp")
            max_frames = str(args.max_frames)

        print(f"{image_path} -> {mp4_path}")

        size=(0,0)
        img_array = []

        for filename in glob.glob(image_path):
            img = cv2.imread(filename)
            height, width, layers = img.shape
            size = (width,height)
            img_array.append(img)

        # print(img_array)
        # fourcc = cv2.VideoWriter_fourcc(*'H264')

        video = cv2.VideoWriter(mp4_path, cv2.VideoWriter_fourcc(*'MP4V'), FRAME_RATE, (width,height))

        for image in img_array:
            video.write(image)

        cv2.destroyAllWindows()
        video.release()
    # out = cv2.VideoWriter('hello.avi',cv2.VideoWriter_fourcc(*'DIVX'), 15, size)
 
    # for i in range(len(img_array)):
    #     out.write(img_array[i])
    # out.release()
    # make video
#     cmd = [
#         'ffmpeg',
#         '-y',
#         '-vcodec', 'png',
#         '-r', str(fps),
#         '-start_number', str(0),
#         '-i', image_path,
#         '-frames:v', max_frames,
#         '-c:v', 'libx264',
#         '-vf',
#         f'fps={fps}',
#         '-pix_fmt', 'yuv420p',
#         # '-crf', '17',
#         # '-preset', 'veryfast',
#         #   '--enable-gpl',
#         # '--enable-libx264',
      
#         mp4_path
#     ]
#     process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
#     stdout, stderr = process.communicate()
#     if process.returncode != 0:
#         print(stderr)
#         raise RuntimeError(stderr)

#     mp4 = open(mp4_path,'rb').read()
#     data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
#     # display.display( display.HTML(f'<video controls loop><source src="{data_url}" download= type="video/mp4"></video>') )

In [32]:


class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__


make_video(dotdict({"max_frames":300, "timestring":20221007125514, "outdir": "../nude/"}))


../nude/v_20221007125514.mp4 pp
../nude/20221007125514_*.png -> ../nude/v_20221007125514.mp4


OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


In [ ]:
experiments = {
    "seed": [9395212, 9395213, 9395214, 9395222],
    "noise_schedule": ["0: (0.02)","0: (0.005)","0: (0.01)","0: (0.05)"]
}

for idx, e in enumerate([*experiments.items()][0][1]):
    e = {k:v[idx] for k,v in experiments.items()}
    dt = datetime.now()
    ts = datetime.timestamp(dt)
    e["timestring"] = str(floor(ts))
    print(e, "variation")
    run(e)

{'seed': 9395212, 'noise_schedule': '0: (0.02)', 'timestring': '1665151101'} variation
reading custom settings from configs/current.txt
{'W': 448, 'H': 512, 'seed': 9395212, 'sampler': 'ddim', 'steps': 200, 'scale': 13, 'ddim_eta': 0, 'dynamic_threshold': None, 'static_threshold': None, 'save_samples': True, 'save_settings': True, 'display_samples': True, 'save_sample_per_step': False, 'show_sample_per_step': False, 'prompt_weighting': False, 'normalize_prompt_weights': True, 'log_weighted_subprompts': False, 'n_batch': 1, 'batch_name': 'whatever', 'filename_format': '{timestring}_{index}_{prompt}.png', 'seed_behavior': 'iter', 'make_grid': False, 'grid_rows': 2, 'outdir': '../nude', 'use_init': True, 'strength': 1, 'strength_0_no_init': True, 'init_image': 'https://i.postimg.cc/mbsMZp47/ivana.jpg', 'use_mask': False, 'use_alpha_as_mask': False, 'mask_file': 'https://i.postimg.cc/LsZTd470/7421-F296-FB94-4241-9-C57-36314-A62-AD28-1-201-a.jpg', 'invert_mask': False, 'mask_brightness_adju

In [ ]:
args_global